In [1]:
import numpy as np
from scipy.integrate import odeint
from tqdm import tqdm
import time
from deap import base, creator, tools, algorithms
import random

# Constants
a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

W2 = 63000
W3 = 20000
W4 = 15000
W5 = 33000
Ws2 = 8000

P980 = 3*10**4  # our power density



def system(state, t, x, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(37*(100-x)-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]




In [2]:

Cr = 8 / 100  # 8%


def calculate_up(up, C2):
    C2 = C2 / 100
    return 3 * up / (2 + (Cr / C2) ** 2)


def calculate_k(k, C2):
    C2 = C2 / 100
    return k * (C2 / Cr) ** 2


def compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51):

    # Compute initial condition
    y = 37 * x
    state0 = [0, y, 0, 0, 0, 0]

    # ODEs
    t = np.arange(0.0, 0.001, 0.000001)

    state = odeint(system, state0, t, args=(x, c1, c2, c3, c4, k31, k41, k51))

    # Compute NIR and blue_total
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total

def objective(exponents):

    # c1, c2, c3, c4, k31, k41, k51 = params
    c1_exp, c2_exp, c3_exp, c4_exp, k31_exp, k41_exp, k51_exp = exponents

    # Converting exponent values to actual values
    c1, c2, c3, c4, k31, k41, k51 = (10**c1_exp, 10**c2_exp, 10**c3_exp,
                                 10**c4_exp, 10**k31_exp, 10**k41_exp, 10**k51_exp)

    up_values = [c1, c2, c3, c4]

    k_values = [k31, k41, k51]


    C2_range = [4, 6, 8, 10, 12, 15, 50]

    # storing new cross relaxation and new up-conversion values
    nested_up_list = []
    nested_k_list = []

    for C2 in C2_range:

        up_sublist = [calculate_up(up, C2) for up in up_values]

        k_sublist = [calculate_k(k, C2) for k in k_values]

        nested_up_list.append(up_sublist)

        nested_k_list.append(k_sublist)

    x_values = [4, 6, 8, 10, 12, 15, 50]

    exp_blue_total = [4.5*10**3, 5.7*10**3, 5.5*10**3, 3.5*10**3, 2*10**3, 1.2*10**3, 0.6*10**3]
    blue_std=[1*10**3, 1.5*10**3, 1*10**3, 1*10**3, 0.5*10**3, 0.5*10**3, 0.25*10**3]

    #blue_weight=[i/j for i, j in zip(exp_blue_total, blue_std)]
    #blue_weight_2=[x/sum(blue_weight) for x in blue_weight]

    exp_NIR = [5*10**3, 8.5*10**3, 6.6*10**3, 7.5*10**3, 6*10**3, 5.5*10**3, 0.6*10**3]
    NIR_std=[1.1*10**3, 2.25*10**3, 0.7*10**3, 1.5*10**3, 1*10**3, 1.7*10**3, 0.25*10**3]

    #NIR_weight=[i/j for i, j in zip(exp_NIR, NIR_std)]
    #NIR_weight_2=[x/sum(NIR_weight) for x in NIR_weight]


    total_error = 0

    for index, x in enumerate(tqdm(x_values)):

        c1, c2, c3, c4 = nested_up_list[index]

        k31, k41, k51 = nested_k_list[index]

        NIR, blue_total = compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51)

        error_NIR = ((NIR - exp_NIR[index])/exp_NIR[index])**2
        error_blue = ((blue_total - exp_blue_total[index])/exp_blue_total[index])**2

        total_error += error_NIR + error_blue


    return total_error, 

In [3]:
# Systematic study ?

alpha1 = 0.1
Average = 0
Std = 3
Mut_p = 0.2
POP_SIZE = 100
CXPB = 0.7
MUTPB = 0.2
NGEN = 60
Cr = 8 / 100

# aim to minimize its values
# How?
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

creator.create("Individual", list, fitness=creator.FitnessMin)

# The toolbox is a container for various functions used in the genetic algorithm
toolbox = base.Toolbox()

# valid_values = [10**i for i in range(-20, 20)]
# toolbox.register("attr_float", random.choice, valid_values) # discrete

valid_values = [i for i in range(-10, 10)]
toolbox.register("attr_float", random.choice, valid_values)


toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=7)

toolbox.register("population", tools.initRepeat, list, toolbox.individual) # create a list of individuals, essentially generating the initial population

# Genetic Operators
toolbox.register("mate", tools.cxBlend, alpha= alpha1)

toolbox.register("mutate", tools.mutGaussian, mu=Average, sigma=Std, indpb=Mut_p)

# Evaluation Function
toolbox.register("evaluate", objective)

toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=POP_SIZE)

fits = list(map(toolbox.evaluate, pop))

for fit, ind in zip(fits, pop):
    ind.fitness.values = fit

for gen in range(NGEN):
    offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)
    # fits = list(map(toolbox.evaluate, offspring))
    for ind in offspring:
        fit = toolbox.evaluate(ind)
        ind.fitness.values = fit

    pop = toolbox.select(offspring, k=len(pop))

top10 = tools.selBest(pop, k=10)



  0%|                                                     | 0/7 [00:00<?, ?it/s]/Users/jayliu/anaconda3/lib/python3.11/site-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
  0%|                                                     | 0/7 [00:00<?, ?it/s]/Users/jayliu/anaconda3/lib/python3.11/site-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Repeated convergence failures (perhaps bad Jacobian or tolerances). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
100%|████████████████████████████████████████████| 7/7 [00:00<00:00, 541.72it/s]


 lsoda--  at t (=r1) and step size h (=r2), the      
       corrector convergence failed repeatedly       
       or with abs(h) = hmin     
      in above,  r1 =  0.2332453570198D-07   r2 =  0.6829085419575D-13


100%|████████████████████████████████████████████| 7/7 [00:00<00:00, 643.14it/s]


 lsoda--  at t (=r1) and step size h (=r2), the      
       corrector convergence failed repeatedly       
       or with abs(h) = hmin     
      in above,  r1 =  0.2332453570198D-07   r2 =  0.6829085419575D-13


100%|████████████████████████████████████████████| 7/7 [00:00<00:00, 247.96it/s]


In [4]:
top10

[[1.751626530604892,
  -1.4280660250879718,
  0.09873413037174358,
  7.324093029358846,
  0.6306939415343724,
  0.7115949375981798,
  -4.729323452643687],
 [1.751626530604892,
  -1.4280660250879718,
  0.09873413037174358,
  7.324093029358846,
  0.6306939415343724,
  0.7115949375981798,
  -4.729323452643687],
 [1.7516369470418383,
  -1.428062421739618,
  0.09871942659953971,
  7.805133779654255,
  0.6306141326938342,
  0.6624873320817501,
  -4.666659079583252],
 [1.7516369470418383,
  -1.428062421739618,
  0.09871942659953971,
  7.805133779654255,
  0.6306141326938342,
  0.6624873320817501,
  -4.666659079583252],
 [1.7516369470418383,
  -1.428062421739618,
  0.09871942659953971,
  7.805133779654255,
  0.6306141326938342,
  0.6624873320817501,
  -4.666659079583252],
 [1.7516369470418383,
  -1.428062421739618,
  0.09871942659953971,
  7.805133779654255,
  0.6306141326938342,
  0.6624873320817501,
  -4.666659079583252],
 [1.751618501587892,
  -1.4280638299520803,
  0.09879361498325662,
  7

In [ ]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML




for i in range(1):

    top_ten_individuals = top10

    # Unzipping the parameters
    c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps = zip(*top_ten_individuals)

    fig = go.Figure()

    # Adding subplots for each parameter
    params = [c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps]
    param_names = ["c1_exp", "c2_exp", "c3_exp", "c4_exp", "k31_exp", "k41_exp", "k51_exp"]

    for param, name in zip(params, param_names):
        fig.add_trace(go.Box(
            y=param,
            name=name
        ))


    fig.update_xaxes(showgrid=True, gridwidth=2, gridcolor='white')
    fig.update_yaxes(showgrid=True, gridwidth=2, gridcolor='white')

    fig.update_layout(
                title=f"Parameter exponents of top ten individuals",
                title_font=dict(size=24, family="Courier New, monospace"),
                title_x=0.5,  # center
                title_y=0.9,
                width=800, height=700,
                yaxis_title="Parameter Exponent Value",
                xaxis_title="Seven Parameter Exponents",
                xaxis=dict(
                title_font=dict(size=22, family="Courier New, monospace"),
                tickfont=dict(size=22, family="Courier New, monospace")
            ),
            yaxis=dict(
                title_font=dict(size=22, family="Courier New, monospace"),
                tickfont=dict(size=22, family="Courier New, monospace")
            )
            )


    fig.show()

In [ ]:
import numpy as np


c1_exp, c2_exp, c3_exp, c4_exp, k31_exp, k41_exp, k51_exp = top10[0]

c1, c2, c3, c4, k31, k41, k51 = (10**c1_exp, 10**c2_exp, 10**c3_exp,
                                 10**c4_exp, 10**k31_exp, 10**k41_exp, 10**k51_exp)

up_values = [c1, c2, c3, c4]

k_values = [k31, k41, k51]

# reference parameter Cr
Cr = 8 / 100  # 8%

# update up-conversion

def calculate_up(up, C2):
    C2 = C2 / 100
    return 3 * up / (2 + (Cr / C2) ** 2)

# update cross-relaxation

def calculate_k(k, C2):
    C2 = C2 / 100
    return k * (C2 / Cr) ** 2


# our data
# Tm concentration from 4% to 50%
C2_range = np.arange(4, 51, 1)



# storing new cross relaxation and new up-conversion values
nested_up_list = []
nested_k_list = []


for C2 in C2_range:

    up_sublist = [calculate_up(up, C2) for up in up_values]

    k_sublist = [calculate_k(k, C2) for k in k_values]

    nested_up_list.append(up_sublist)

    nested_k_list.append(k_sublist)


In [ ]:
import numpy as np
from scipy.integrate import odeint
from tqdm import tqdm
import time
import random

# Constants
a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

W2 = 63000
W3 = 20000
W4 = 15000
W5 = 33000
Ws2 = 8000

P980 = 3*10**4  # our PD



def system(state, t, x, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(37*(100-x)-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]




In [ ]:
from scipy.integrate import odeint
from tqdm import tqdm

def compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51):

    # Compute initial condition
    y = 37 * x
    state0 = [0, y, 0, 0, 0, 0]

    # ODEs
    t = np.arange(0.0, 0.001, 0.000001)

    state = odeint(system, state0, t, args=(x, c1, c2, c3, c4, k31, k41, k51))

    # Compute NIR and blue_total
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total


x_values = list(np.arange(4, 51, 1))  # Range of C2 values from 2% to 40%
NIR_values = []
blue_total_values = []


for index, x in enumerate(tqdm(x_values)):

    c1, c2, c3, c4 = nested_up_list[index]

    k31, k41, k51 = nested_k_list[index]

    NIR, blue_total = compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51)

    # Store the results
    NIR_values.append(NIR)
    blue_total_values.append(blue_total)



sum=[i+j for i, j in zip(NIR_values, blue_total_values)]


In [ ]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML

fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=x_values + x_values[::-1], y=NIR_values, mode='lines+markers', name='NIR simulation', marker=dict(color='rgb(170, 68, 153)')))
fig1.add_trace(go.Scatter(x=x_values + x_values[::-1], y=blue_total_values, mode='lines+markers', name='Blue simulation', marker=dict(color='#0099C6')))
fig1.add_trace(go.Scatter(x=x_values, y=sum, mode='lines+markers', name='Total', marker=dict(color='rgb(102, 102, 102)')))


# x=[4, 6, 8, 10, 12, 15, 50]
# x_shift=[i+4 for i in x]

exp_blue_total = [4.5*10**3, 5.7*10**3, 5.5*10**3, 3.5*10**3, 2*10**3, 1.2*10**3, 0.6*10**3]
exp_NIR = [5*10**3, 8.5*10**3, 6.6*10**3, 7.5*10**3, 6*10**3, 5.5*10**3, 0.6*10**3]
exp_sum = [i+j for i, j in zip(exp_blue_total, exp_NIR)]


NIR_std = [1.1*10**3, 2.25*10**3, 0.7*10**3, 1.5*10**3, 1*10**3, 1.7*10**3, 0.25*10**3]
blue_std = [1*10**3, 1.5*10**3, 1*10**3, 1*10**3, 0.5*10**3, 0.5*10**3, 0.25*10**3]
sum_std = [2.1*10**3, 3.5*10**3, 1.5*10**3, 2.5*10**3, 1.4*10**3, 2.1*10**3]

fig1.add_trace(go.Scatter(x= [4, 6, 8, 10, 12, 15, 50], y= exp_NIR, mode='markers', name='NIR_exp', marker=dict(color='rgb(170, 68, 153)', size=10), error_y=dict(type='data', array=NIR_std, visible=True,color='rgb(170, 68, 153)', width=3, thickness=1.5 )))
fig1.add_trace(go.Scatter(x= [4, 6, 8, 10, 12, 15, 50], y= exp_blue_total, mode='markers', name='Blue_exp', marker=dict(color='#0099C6', size=10), error_y=dict(type='data', array=blue_std, visible=True, color='#0099C6', width=3,thickness=1.5 )))
fig1.add_trace(go.Scatter(x= [4, 6, 8, 10, 12, 15, 50], y= exp_sum, mode='markers', name='Sum_exp', marker=dict(color='rgb(102, 102, 102)', size=10), error_y=dict(type='data', array=sum_std, visible=True, color='rgb(102, 102, 102)', width=3,thickness=1.5 )))



# Update y-axis to show values in units of 10^4

y_ticks = [i for i in range(int(min(min(exp_sum), min(blue_total_values)) / 1e3),
                            int(max(max(exp_sum), max(blue_total_values)) / 1e3) + 1)]

fig1.update_yaxes(tickvals=[tick * 1e3 for tick in y_ticks],
                 ticktext=[f"{tick} × 10^3" for tick in y_ticks])


fig1.update_xaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white', dtick=2)
fig1.update_yaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white')


fig1.update_layout(
    title_text='NaYbF4: (1-x)% Yb, x% Tm',  # Main Title
    title_font=dict(size=24, family="Arial, sans-serif", color='royalblue'),
    width=1200, height=900,
    xaxis_title='Tm concentration(%)',
    yaxis_title='Intensity(kpps)',
    title_x=0.42,  # Center the main title
    title_y=0.95,  # Adjust the vertical position of the main title
    showlegend=True, margin=dict(l=20, r=20, b=20, t=70),
    legend=dict(
        font=dict(
            size=20,
            family="Arial, sans-serif"
        )
    )
    )

